In [1]:
from dask.distributed import Client, LocalCluster
from dask import array as darr

In [2]:
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit=1e9)
# cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit=250e6)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:45915 Dashboard: /proxy/8787/status,Cluster Workers: 1 Cores: 4 Memory: 1000.00 MB


In [3]:
x = darr.random.normal(
    size=(300_000_000, 3),
    chunks=(5_000_000, 1)
)

In [ ]:
x

In [ ]:
# x = x.persist()

In [ ]:
xm = x.max(axis=-1).mean()

In [ ]:
xm

In [ ]:
xm.compute()

In [ ]:
x = darr.random.normal(
    size=(100_000_000, ),
    chunks=(1_000_000, )
)
display(x)

In [ ]:
%%time

x.mean().compute()

In [ ]:
x = darr.random.normal(
    size=(100_000_000, ),
    chunks=(1_000, )
)
display(x)

In [ ]:
%%time

x.mean().compute()

In [ ]:
x = darr.random.normal(
    size=(10_000, 10_000),
    chunks=(100, 10_000)
)
display(x)

In [ ]:
x.mean(axis=1).mean(axis=-1).compute()

In [4]:
import xarray as xr

In [5]:
ds = xr.DataArray(
    darr.random.normal(size=(12_500_000, 1), chunks=(12_500_000, 1))
).to_dataset(name="test")
ds 

<xarray.Dataset>
Dimensions:  (dim_0: 12500000, dim_1: 1)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    test     (dim_0, dim_1) float64 dask.array<chunksize=(12500000, 1), meta=np.ndarray>

In [6]:
ds.to_netcdf("test.nc")

In [12]:
ds = xr.open_mfdataset(
    "test.nc", chunks={"dim_0": 500_000}
)
ds

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


<xarray.Dataset>
Dimensions:  (dim_0: 12500000, dim_1: 1)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    test     (dim_0, dim_1) float64 dask.array<chunksize=(500000, 1), meta=np.ndarray>

In [14]:
ds = ds.persist()

In [23]:
x = darr.ones(shape=(4_000, 4_000), chunks=(4_000, 100))
x

dask.array<ones, shape=(4000, 4000), dtype=float64, chunksize=(4000, 100), chunktype=numpy.ndarray>

In [24]:
y = x.rechunk((100, 1_000))
y

dask.array<rechunk-merge, shape=(4000, 4000), dtype=float64, chunksize=(100, 1000), chunktype=numpy.ndarray>

In [25]:
y.std(axis=-1).compute()

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
y = x.rechunk((100, 4_000))
y

dask.array<rechunk-merge, shape=(4000, 4000), dtype=float64, chunksize=(100, 4000), chunktype=numpy.ndarray>

In [27]:
y.std(axis=-1).compute()

array([0., 0., 0., ..., 0., 0., 0.])